In [ ]:
from gen_ai_hub.proxy.langchain import init_llm
from hana_ai.agents.chatbot_with_memory import ChatbotWithMemory
from hana_ml import dataframe
from hana_ai.tools.toolkit import HANAMLToolkit
from hana_ml.algorithms.pal.utility import Settings
url, port, user, pwd = Settings.load_config("../../config/e2edata.ini")
cc = dataframe.ConnectionContext(url, port, user, pwd)
llm = init_llm('gpt-4-32k', temperature=0.0, max_tokens=600)
tools = HANAMLToolkit(cc, used_tools='ts_outlier_detection').get_tools()
chatbot = ChatbotWithMemory(llm=llm, tools=tools, session_id='ts_outlier_test_hou', n_messages=10)

In [3]:
import numpy as np
import pandas as pd
np.random.seed(23)
X = range(256)
x = np.arange(256) / 64
Y = x + 0.8 * np.sin(x * 2 * np.pi) + np.random.normal(scale=0.1, size=256)
Y[74] = Y[74] + 0.6 #1st outlier added at index 74
Y[126] = Y[126] - 0.8 #2nd outlier added at index 126
data = pd.DataFrame(dict(ID=X, Y=Y))
from hana_ml.dataframe import create_dataframe_from_pandas
ts_outlier_df = create_dataframe_from_pandas(cc, data,
                                             "SIM_TSOUTLIER_DATA_TBL",
                                             force=True)

100%|██████████| 1/1 [00:00<00:00,  2.78it/s]


In [4]:
chatbot.chat("Find the collection of outliers in SIM_TSOUTLIER_DATA_TBL")

"To find the outliers in the 'SIM_TSOUTLIER_DATA_TBL' table, I need some additional information. Could you please provide the key and endog of the dataset?"

In [5]:
chatbot.chat("Yes, the key is ID and endog is Y")

'{"outliers": [74, 126], "Auto Outlier Method": "seasonal decomposition and super smoother", "DETECT_INTERMITTENT_TS": "0", "INTERMITTENT_FLAG": "0", "REMOVE_LEADING_AND_TRAILING_ZEROS": "0", "TAKING_LOG_FLAG": "0", "Periods": "63", "ACFs": "0.734877", "OutlierNum": "2", "Mean": "-0.00382868", "Standard Deviation": "0.0949135", "HandleZero": "0"}'